In [1]:
import math
import numpy as np
import scipy.sparse as sp

import igl
import meshplot as mp
from meshplot import plot, subplot, interact

from math import sqrt

In [2]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")

v_cam, f_cam  = igl.read_triangle_mesh("./data/camel_head.off")

tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

print(f.shape)

(756, 3)


In [3]:
def align_field_hard(V, F, hard_id, hard_value):
    """
    This function creates a smooth vector field by formulating the problem with hard constraints
    """
    
    # reordering the sequences of faces so that the faces with constraints are
    # at the end
    F_constrained = F[hard_id] # selecting the faces with constraints
    F_free = np.delete(F, hard_id, axis = 0) # selecting the faces without constraints
    F_str = np.vstack((F_free, F_constrained)) # restacking the face array

    # Computing the adjacency list
    TT_str, _ = igl.triangle_triangle_adjacency(F_str)

    # Edges
    e1 = V[F_str[:, 1], :] - V[F_str[:, 0], :]
    e2 = V[F_str[:, 2], :] - V[F_str[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F_str.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT_str[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F_str[f, (ei+1)%3], :] - V[F_str[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
    
    d = np.zeros(hard_id.shape[0], dtype=np.complex)
    
    for ci in range(hard_id.shape[0]):
        f = len(F_free) + ci
        v = hard_value[ci, :]
        
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        d[ci] = c
    
    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    A = A.H @ A
    
    # variable elimination
    A_tilde = A[0:len(F_free), 0:len(F_free)]
    A_fc = A[0:len(F_free), len(F_free):]
    b_tilde = - A_fc @ d
    
    u_fr = sp.linalg.spsolve(A_tilde, b_tilde)
    u = np.hstack((u_fr, d))
    
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R, F_str
    

In [4]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + R * avg)
    
    return p

In [5]:
R, f_str = align_field_hard(v, f, cf, c)
np.savetxt("interpolated_field_hard", R)

In [6]:
cf_str = np.arange(len(f_str) - len(cf), len(f_str)) # the last n elements are constrained elements
plot_mesh_field(v, f_str, R, cf_str)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [7]:
# flattening R from the previous question to be used in least squares
u = R.flatten("F")
# computing gradients
G = igl.grad(v, f_str)
# creating the sparese weight vector with areas
wt = sp.diags(np.tile(igl.doublearea(v, f), 3))
# solving the problem but setting a hard constraint of zero for the last scalar gradient value
s = sp.linalg.spsolve((G.T@wt@G)[0:-1,0:-1], (G.T@wt@u)[0:-1])
# restacking to get the entire gradient flattened vector with constrained component as well
s = np.hstack((s, 0))
np.savetxt("scalar_function", s)
# checking if the solution is close to desired value
gt = G@s
print(np.linalg.norm(gt-u))
# reshaping to for gradient vector (# F * 3)
gt_vec = np.hstack((gt[0:len(f_str):,None], np.hstack((gt[len(f_str):2*len(f_str):,None], gt[2*len(f_str)::,None]))))

7.357290355143346


In [8]:
def plot_scalar_field_with_grad(V, F, R, R_des, constrain_faces, cmap):
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=cmap)
    # the desired vector field
    p.add_lines(B, B + R_des * avg, shading = {"line_color":"red"})
    # graidents of the scalar field
    p.add_lines(B, B + R * avg)
    
    return p

In [9]:
p = plot_scalar_field_with_grad(v, f_str, gt_vec, R, cf_str, s)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [10]:
def harmonic_parameterization(v, f):
    """
    This function computes the harmonic parametrization of a mesh
    """
    ## Find the open boundary
    bnd = igl.boundary_loop(f)

    ## Map the boundary to a circle, preserving edge proportions
    bnd_uv = igl.map_vertices_to_circle(v, bnd)

    ## Harmonic parametrization for the internal vertices
    uv = igl.harmonic_weights(v, f, bnd, bnd_uv, 1)

    return uv

def plot_gradient(V, F, R, p  = None):
    # Highlight in red the constrained faces
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)
    if p == None:
        p = mp.plot(V, F)
    
    p.add_lines(B, B + R * avg)
    
    return p

# computing harmonic parameterization for the cylinder mesh
uv = harmonic_parameterization(v, f_str)
# ploting the uv map and mesh with the v scalar values as color map
p = subplot(v, f_str, c = uv[:,1], s=[1, 2, 0], shading={"wireframe": True, "flat": False})
subplot(uv, f_str, shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p, c = uv[:,1])

# UV map with checked board
p = subplot(v, f_str, uv = uv, s=[1, 2, 0], shading={"wireframe": False, "flat": False})
subplot(uv, f_str, shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p, uv = uv)



In [11]:
# computing gradient of the v mapping
G = igl.grad(v, f_str)
gt = G @ uv[:,1]
# converting to vector form for ploting
gt_vec = np.hstack((gt[0:len(f):,None], np.hstack((gt[len(f):2*len(f):,None], gt[2*len(f)::,None]))))

# replacing v map with smooth user-guided vector field
uv_modified = uv.copy()
uv_modified[:,1] = s
gt = G @ s
gt_vec_modified = np.hstack((gt[0:len(f):,None], np.hstack((gt[len(f):2*len(f):,None], gt[2*len(f)::,None]))))

# UV map with checked board
p_modified = subplot(v, f_str, uv = uv_modified, s=[1, 2, 0], shading={"wireframe": False, "flat": False})
subplot(uv_modified, f_str, shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p_modified, uv = uv_modified)
plot(uv_modified, f_str, shading={"wireframe": True, "flat": False}, c = uv[:,1])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.82…

In [12]:
# plotting 
p = plot(v, f_str, c = uv[:,1], shading={"wireframe": False, "flat": True})
plot_gradient(v, f_str, gt_vec, p)

p = plot(v, f_str, c = uv_modified[:,1], shading={"wireframe": True, "flat": True})
plot_gradient(v, f_str, gt_vec_modified, p)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [13]:
def plot_flipped_triangles(V, F, flipped_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(F)
    col[flipped_faces, 1:] = 0
    p = mp.plot(V, F, c=col, shading={"wireframe": True, "flat": True})
    
    return p

# determining which triangles are flipped 
is_flipped = np.zeros(len(f_str))
k = 0
for i in range(len(f_str)):
    e1 = uv[f_str[i][1]] - uv[f_str[i][0]]
    e2 = uv[f_str[i][2]] - uv[f_str[i][0]]
    
    # computing normal to see if vertices are counter clockwise
    n = np.cross(e1, e2)

    e1_mod = uv_modified[f_str[i][1]] - uv_modified[f_str[i][0]]
    e2_mod = uv_modified[f_str[i][2]] - uv_modified[f_str[i][0]]
    
    # computing normal with new modified uv mapping
    n_mod = np.cross(e1_mod, e2_mod)
    
    # a face is flipped if product is negative mostly because the original triangle
    # the vertices will be counter clockwise (n = 1), while the other would be -1 (sign)
    is_flipped[i] = np.sign(n*n_mod)

    
flipped_triangles = np.where(is_flipped == -1)
np.savetxt("flipped_triangles", flipped_triangles)

p = plot_flipped_triangles(v, f_str, flipped_triangles)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…